<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting-up-imports" data-toc-modified-id="Setting-up-imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting up imports</a></span></li><li><span><a href="#Setting-up-Constant-Hyperparameters" data-toc-modified-id="Setting-up-Constant-Hyperparameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up Constant Hyperparameters</a></span></li><li><span><a href="#Setting-up-Parameters-and-Functions-for-Training" data-toc-modified-id="Setting-up-Parameters-and-Functions-for-Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up Parameters and Functions for Training</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters-Search-Space" data-toc-modified-id="Hyperparameters-Search-Space-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hyperparameters Search Space</a></span></li><li><span><a href="#Creating-the-training-function" data-toc-modified-id="Creating-the-training-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating the training function</a></span></li><li><span><a href="#Creating-the-evaluation-function" data-toc-modified-id="Creating-the-evaluation-function-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Creating the evaluation function</a></span></li></ul></li><li><span><a href="#Running-the-training" data-toc-modified-id="Running-the-training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running the training</a></span><ul class="toc-item"><li><span><a href="#Loading-data-for-training" data-toc-modified-id="Loading-data-for-training-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Loading data for training</a></span></li><li><span><a href="#Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm" data-toc-modified-id="Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Configuring the Tuner with a Scheduler and a Search Algorithm</a></span></li><li><span><a href="#Running-the-Tuner" data-toc-modified-id="Running-the-Tuner-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Running the Tuner</a></span></li></ul></li><li><span><a href="#Evaluating-the-best-Results" data-toc-modified-id="Evaluating-the-best-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluating the best Results</a></span></li></ul></div>

# Setting up imports

In [1]:
import os
from itertools import product

import torch
from torch.nn import CrossEntropyLoss, Sequential
from torch.nn.functional import normalize
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision.transforms import CenterCrop, Resize, GaussianBlur
# from torchvision.transforms.functional import invert

import ray
from ray import tune
from ray.air import session, RunConfig, CheckpointConfig
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.search.hyperopt import HyperOptSearch
# from ray.tune.search.optuna import OptunaSearch
# from ray.tune.search import ConcurrencyLimiter


from dataset import POCDataReader, data_augment_, POCDataset
from metrics import Metrics, EvaluationMetrics
from models import UNet
from loss import *
from pipelines import InputPipeline, SequenceFilters, SumFilters
from pipelines.filters import *
from train import training_loop, validation_loop
from train_tqdm import evaluation_loop


# Setting up Constant Hyperparameters

In [2]:
EPOCHS = 20
NUM_SAMPLES = 1

NUM_AUGMENT = 1

LOAD_DATA_ON_GPU = False
GPUS_PER_TRIAL = 1
CPUS_PER_TRIAL = 4

##### Selecting Cuda device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Setting up Parameters and Functions for Training

## Hyperparameters Search Space

##### Preload Losses Functions

In [4]:
pixel_loss_list = [
    CrossEntropyLoss(weight=torch.tensor([.3, .7])),
    FocalLoss(weight=torch.tensor([.3, .7]), gamma=2),
]

volume_loss_list = [
    JaccardLoss(),
    TverskyLoss(alpha=0.3, beta=0.7),
    FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=2),
]

loss_list = []
for ploss in pixel_loss_list:
    loss_list.append(PixelLoss(pixel_loss=ploss, volume_loss=None))
for vloss in volume_loss_list:
    loss_list.append(VolumeLoss(pixel_loss=None, volume_loss=vloss))
for (ploss, vloss) in product(pixel_loss_list, volume_loss_list):
    loss_list.append(CombinedLoss(loss1=ploss, loss2=vloss, ratio=0.3))
    loss_list.append(BorderedLoss(border_loss=ploss, volume_loss=vloss, ratio=0.7))

##### Preload Filter

In [5]:
filter_list = [normalize] #, invert]

layer_list = [
    SobelFilter(),
    LaplacianFilter(),
    FrangiFilter(),
    SatoFilter(),
    SumFilters(FrangiFilter(), SatoFilter()),
    SkeletonFilter(SequenceFilters(SumFilters(FrangiFilter(), SatoFilter()), CrackBinaryFilter())),
]

pipeline_list = []
for f, l in product(filter_list, layer_list):
    pipeline_list.append(InputPipeline(transformer=f, layer_transformer=l))

##### Search Space

In [6]:
search_space = {
    "Network": UNet, #tune.grid_search([Unet, DeepCrack, SubUNet]),
    "Optimizer": Adam,

    "Learning Rate": 1e-4, #tune.loguniform(1e-6, 1e-4),
    "Batch Size": 4,           #tune.qrandint(2, 8, 2),

    "Loss Function": tune.grid_search(loss_list),

    "Negative Mining": True, #tune.choice([True, False]),
    "Smooth Labeling": False, #tune.choice([True, False]),

    "Input Pipeline": tune.grid_search(pipeline_list),
}

## Creating the training function

In [7]:
def train(config, train_data, val_data):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inpip = config["Input Pipeline"]
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    train_dataset = POCDataset(
        train_data,
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(400, 400))),
        target_transform= Sequential(
            GaussianBlur(kernel_size=3, sigma=0.7),
            CenterCrop(size=(480, 480)),
            Resize(size=(400, 400)),
        ) if config["Smooth Labeling"] else Sequential(
            CenterCrop(size=(480, 480)),
            Resize(size=(400, 400)),
        ),
        negative_mining=config["Negative Mining"],
        load_on_gpu=LOAD_DATA_ON_GPU)
    train_dataset.precompute_transform()

    if LOAD_DATA_ON_GPU:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
        )
    else:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    val_dataset = POCDataset(
        val_data, 
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(400, 400))),
        target_transform=Sequential(CenterCrop(size=(480, 480)), Resize(size=(400, 400))),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    val_dataset.precompute_transform()
    
    if LOAD_DATA_ON_GPU:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True)
    else:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    model = config["Network"](n_channels=inpip.nb_channel, n_classes=2)
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    loss_fn = config["Loss Function"].to(device)
    optimizer = config["Optimizer"](model.parameters(), lr=config["Learning Rate"], betas=(0.9, 0.99))
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state, scheduler_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        lr_scheduler.load_state_dict(scheduler_state)

    train_metrics = Metrics(
        buffer_size=len(training_dataloader),
        mode="Training",
        hyperparam=config,
        device=device)

    val_metrics = Metrics(
        buffer_size=len(validation_dataloader),
        mode="Validation",
        hyperparam=config,
        device=device)


    for epoch in range(1, EPOCHS+1):  # loop over the dataset multiple times
        training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
        validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("model", exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict()), "model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("model")
        session.report(metrics=val_metrics.get_metrics(epoch), checkpoint=checkpoint)

    train_metrics.close_tensorboard()
    val_metrics.close_tensorboard()


## Creating the evaluation function

In [8]:
def evaluate(test_data, result):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    inpip = config["Input Pipeline"]
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    test_dataset = POCDataset(
        test_data,
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(400, 400))),
        target_transform=Sequential(CenterCrop(size=(480, 480)), Resize(size=(400, 400))),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    
    if LOAD_DATA_ON_GPU:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
    else:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2*CPUS_PER_TRIAL, pin_memory=True, pin_memory_device=device)

    best_trained_model = result.config["Network"](n_channels=inpip.nb_channel, n_classes=2, bilinear=True, crop=False).to(device)

    checkpoint_path = os.path.join(result.checkpoint.to_directory(), "checkpoint.pt")
    model_state, _, _ = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    test_metrics = EvaluationMetrics(
        buffer_size=len(evaluation_dataloader),
        hyperparam=result.config,
        epochs=result.metrics["Epoch"],
        device=device)

    evaluation_loop(dataloader=evaluation_dataloader, model=best_trained_model, metric=test_metrics, device=device)


# Running the training

## Loading data for training

In [9]:
data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, verbose=True)
train_data, val_data, test_data = data_reader.split([0.7, 0.1, 0.2])
data_augment_(train_data, n=NUM_AUGMENT, load_on_gpu=False, verbose=True)

Loading dataset into RAM:   0%|          | 0/2744 [00:00<?, ?it/s]

	- Loading done, RAM used: 5.40GiB / free: 34.27GiB / total: 62.73GiB
	- Got a total of 2744 images.


Expending the dataset 1 more times:   0%|          | 0/1920 [00:00<?, ?it/s]

	- Augmentation done, RAM used: 8.16GiB / free: 31.51GiB / total: 62.73GiB
	- Got 1920 new images and a total of 3840 images.


## Configuring the Tuner with a Scheduler and a Search Algorithm

In [10]:
# scheduler = ASHAScheduler(max_t=EPOCHS, grace_period=2, reduction_factor=2)
# search_algo = HyperOptSearch()
# search_algo = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="CrackIoU",
    mode="max",
    num_samples=NUM_SAMPLES,
#     scheduler=scheduler,
#     search_alg=search_algo,
)

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train, train_data=train_data, val_data=val_data),
        resources={"cpu": CPUS_PER_TRIAL, "gpu": GPUS_PER_TRIAL}),
    tune_config=tune_config,
    param_space=search_space,
    run_config=RunConfig(
        local_dir="~/Documents/POC-Project/ray_results",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="CrackIoU",
            checkpoint_score_order="max")))

## Running the Tuner

In [11]:
results = tuner.fit()

2023-04-07 12:34:05,842	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
train_12206_00000,2023-04-07_14-11-57,a6b2cb5999574c94930a0ce71be3c4ef,piai-Precision-Tower-7910,141.223.107.22,16771,1680844317,12206_00000
train_12206_00001,2023-04-07_14-24-30,3341b9c1035f4e63a28414608ccda31f,piai-Precision-Tower-7910,141.223.107.22,16816,1680845070,12206_00001
train_12206_00002,2023-04-07_16-02-59,a6b2cb5999574c94930a0ce71be3c4ef,piai-Precision-Tower-7910,141.223.107.22,16771,1680850979,12206_00002
train_12206_00003,2023-04-07_16-22-27,3341b9c1035f4e63a28414608ccda31f,piai-Precision-Tower-7910,141.223.107.22,16816,1680852147,12206_00003
train_12206_00004,2023-04-07_18-05-46,a6b2cb5999574c94930a0ce71be3c4ef,piai-Precision-Tower-7910,141.223.107.22,16771,1680858346,12206_00004
train_12206_00005,2023-04-07_18-40-11,3341b9c1035f4e63a28414608ccda31f,piai-Precision-Tower-7910,141.223.107.22,16816,1680860411,12206_00005
train_12206_00006,2023-04-07_19-44-00,a6b2cb5999574c94930a0ce71be3c4ef,piai-Precision-Tower-7910,141.223.107.22,16771,1680864240,12206_00006
train_12206_00007,2023-04-07_20-30-45,3341b9c1035f4e63a28414608ccda31f,piai-Precision-Tower-7910,141.223.107.22,16816,1680867045,12206_00007
train_12206_00008,2023-04-07_21-35-30,a6b2cb5999574c94930a0ce71be3c4ef,piai-Precision-Tower-7910,141.223.107.22,16771,1680870930,12206_00008
train_12206_00009,2023-04-07_22-28-55,3341b9c1035f4e63a28414608ccda31f,piai-Precision-Tower-7910,141.223.107.22,16816,1680874135,12206_00009


2023-04-07 14:11:57,700	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SobelFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-07 14:24:30,765	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-07 16:03:00,167	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): FrangiFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-07 16:22:27,309	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Inpu

2023-04-08 18:02:18,800	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SkeletonFilter
  )
), 'Loss Function': VolumeLoss(
  (volume_loss): FocalTverskyLoss
)}
2023-04-08 18:56:29,492	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SobelFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): JaccardLoss
)}
2023-04-08 19:42:53,969	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): JaccardLoss
)}
2023-04-08 20:57:22,230	INFO tensorboardx.py:267 -- Removed the following hyperparamete

2023-04-09 19:05:22,110	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): FocalTverskyLoss
)}
2023-04-09 19:59:08,093	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): FrangiFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): FocalTverskyLoss
)}
(raylet) [2023-04-09 19:59:11,939 E 16313 16313] (raylet) node_manager.cc:3040: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: f23f2b0e98a5dbe3f362b7b0159eafdd21a7c11a8d8772aa42c80af4, IP: 141.223.107.22) over the last time period. To see more information about the Workers killed on this node, use

...

(train pid=5796) /home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
(train pid=5796)   return torch._C._cuda_getDeviceCount() > 0
2023-04-10 00:06:51,723	ERROR trial_runner.py:1062 -- Trial train_12206_00090: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=5796, ip=141.223.107.22, repr=train)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint

(train pid=6292) /home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
(train pid=6292)   return torch._C._cuda_getDeviceCount() > 0
2023-04-10 00:24:11,725	ERROR trial_runner.py:1062 -- Trial train_12206_00093: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=6292, ip=141.223.107.22, repr=train)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint

(train pid=7144) /home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
(train pid=7144)   return torch._C._cuda_getDeviceCount() > 0
2023-04-10 00:53:46,348	ERROR trial_runner.py:1062 -- Trial train_12206_00096: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=7144, ip=141.223.107.22, repr=train)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint

(train pid=7531) /home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
(train pid=7531)   return torch._C._cuda_getDeviceCount() > 0
2023-04-10 01:11:04,328	ERROR trial_runner.py:1062 -- Trial train_12206_00099: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=7531, ip=141.223.107.22, repr=train)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint

2023-04-10 01:42:58,767	ERROR trial_runner.py:1062 -- Trial train_12206_00101: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=7920, ip=141.223.107.22, repr=train)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint
    return self._trainable_func(
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 654, in _trainable_func
    output = fn()
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/util.py", line 406, in _inner
    return inner(config, checkpoint_dir=None)
  File "/home/piai/anaconda3/envs/POC-env/lib/python3.10/site-packages/ray/tune/trainable/util.py", line 398, in inn

# Evaluating the best Results

In [12]:
best_result = results.get_best_result(metric="CrackIoU", mode="max", scope="all")  # Get best result object
print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(best_result.metrics["Loss"]))
print("Best trial final validation CrackIoU: {}".format(best_result.metrics["CrackIoU"]))

for result in results:
    evaluate(test_data=test_data, result=result)

Best trial config: {'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Learning Rate': 0.0001, 'Batch Size': 4, 'Loss Function': BorderedLoss(
  (border_loss): CrossEntropyLoss()
  (volume_loss): JaccardLoss
), 'Negative Mining': True, 'Smooth Labeling': False, 'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SobelFilter
  )
)}
Best trial final validation loss: 0.08369708806276321
Best trial final validation CrackIoU: 0.7667993903160095


NameError: name 'config' is not defined